# Integrantes Sala 7
- Guillermo Jair Montiel Juárez
- Juan Nicolás Pinilla Morales
- Melesio Reyes Perez
- Daniel Herrera Caballero
- Juan David de la Cruz Martinez

In [12]:
!pip install gradio transformers --quiet

In [13]:
import gradio as gr
from transformers import pipeline


In [14]:
qa_pipeline = pipeline(
    "question-answering",
    model="NchuNLP/Chinese-Question-Answering",
    tokenizer="NchuNLP/Chinese-Question-Answering"
)


Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


In [15]:
# Longformer acepta hasta ~4000 tokens, así que dividiremos el texto si es más largo
def dividir_texto(texto, max_longitud=3900):
    oraciones = texto.split(". ")
    bloques = []
    actual = ""
    for oracion in oraciones:
        if len(actual) + len(oracion) < max_longitud:
            actual += oracion + ". "
        else:
            bloques.append(actual.strip())
            actual = oracion + ". "
    if actual:
        bloques.append(actual.strip())
    return bloques


In [16]:
def responder_largo(pregunta, archivo):
    try:
        if archivo is None:
            return "Por favor sube un archivo .txt con contenido válido."

        # Leer archivo según tipo
        if hasattr(archivo, "name"):
            with open(archivo.name, encoding="utf-8") as f:
                contexto = f.read()
        else:
            return "⚠️ No se pudo leer el archivo."

        if not pregunta.strip():
            return "Escribe una pregunta válida."

        bloques = [contexto] if len(contexto) < 3000 else dividir_texto(contexto)
        respuestas = []

        for bloque in bloques:
            resultado = qa_pipeline(question=pregunta, context=bloque, top_k=3)
            # Aseguramos que sea una lista (algunos modelos devuelven dict si top_k=1)
            if isinstance(resultado, dict):
                resultado = [resultado]
            for r in resultado:
                if r['score'] >= 0.001:  # solo respuestas confiables
                    respuestas.append((r['answer'], round(r['score'], 3)))

        if not respuestas:
            return "⚠️ No se encontró una respuesta con confianza suficiente."

        # Ordenar por score descendente y mostrar top 3
        respuestas.sort(key=lambda x: x[1], reverse=True)
        texto_respuestas = "\n".join(
            [f"• {resp} (confianza: {score})" for resp, score in respuestas[:3]]
        )
        return texto_respuestas

    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [17]:
# Interfaz Gradio
iface = gr.Interface(
    fn=responder_largo,
    inputs=[
        gr.Textbox(label="Pregunta"),
        gr.File(label="Carga un archivo .txt", file_types=[".txt"])
    ],
    outputs="text",
    title="QA con Longformer para documentos largos",
    description="Este sistema utiliza Longformer para responder preguntas en archivos largos (.txt). Divide automáticamente el contenido si supera los 4000 tokens."
)

In [18]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b74b77b23d194a54d8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
